In [9]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
# Warning is Gensim related

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [11]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
doclist_txt[:5]

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

24-Jan-2017 15:27:44 INFO preprocessing: Creating document list from CSV files ...
24-Jan-2017 15:27:44 DEBUG preprocessing: 16 entries in document list.


['corpus_csv/Howard_GodsoftheNorth.txt.csv',
 'corpus_csv/Poe_EurekaAProsePoem.txt.csv',
 'corpus_csv/Poe_TheMasqueoftheRedDeath.txt.csv',
 'corpus_csv/Poe_ThePurloinedLetter.txt.csv',
 'corpus_csv/Howard_ShadowsintheMoonlight.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [12]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

#### Corpus laden

In [17]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [18]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[:1]

[['a',
  'scandal',
  'in',
  'bohemia',
  'a',
  'conan',
  'doyle',
  'i',
  'to',
  'sherlock',
  'holmes',
  'she',
  'is',
  'always',
  'the',
  'woman',
  'i',
  'have',
  'seldom',
  'heard',
  'him',
  'mention',
  'her',
  'under',
  'any',
  'other',
  'name',
  'in',
  'his',
  'eyes',
  'she',
  'eclipses',
  'and',
  'predominates',
  'the',
  'whole',
  'of',
  'her',
  'sex',
  'it',
  'was',
  'not',
  'that',
  'he',
  'felt',
  'any',
  'emotion',
  'akin',
  'to',
  'love',
  'for',
  'irene',
  'adler',
  'all',
  'emotions',
  'and',
  'that',
  'one',
  'particularly',
  'were',
  'abhorrent',
  'to',
  'his',
  'cold',
  'precise',
  'but',
  'admirably',
  'balanced',
  'mind',
  'he',
  'was',
  'i',
  'take',
  'it',
  'the',
  'most',
  'perfect',
  'reasoning',
  'and',
  'observing',
  'machine',
  'that',
  'the',
  'world',
  'has',
  'seen',
  'but',
  'as',
  'a',
  'lover',
  'he',
  'would',
  'have',
  'placed',
  'himself',
  'in',
  'a',
  'false'

#### Create Dictionaries

In [19]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [20]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [21]:
sparse_bow

0
doc_id token_id    
0      12288      1
       16386      2
       16885      1
       20493      1
       4110       3
       17069      1
       16401      2
       20499      8
       16404      1
       20501      1
       23         1
       16409      1
       4127       2
       8229       4
       4138       1
       16428      3
       12334      2
       8          1
       20733      5
       4148       1
       12341      3
       8248       4
       4155       3
       12349      1
       4160       1
       17240      1
       16450      1
       8260       2
       12359      2
       4169       5
...              ..
16     8135       1
       4043       2
       8140       1
       20429      2
       20430      1
       16335      2
       20434      1
       8149       1
       4056       1
       20441      3
       24538      1
       9551       1
       24542      1
       20447      1
       8160       1
       24545      2
       4067      27
       12260      3
       16359      9
       12264      1
       8170      35
       12268      2
       4077       1
       20463      2
       20464      1
       20466      5
       16371      1
       8189       1
       8190       1
       8191       2

[58745 rows x 1 columns]

##### Speichern der Market Matrix

In [ ]:
num_docs = max(sparse_df.index.get_level_values("doc_id"))
num_types = max(sparse_df.index.get_level_values("token_id"))
sum_counts = sum(sparse_df[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    #f.write(header_string)
    sparse_df.to_csv( f, sep = ' ', header = None)

#### Remove Features

In [22]:
import os.path
basepath =  os.path.abspath(os.path.join(".", os.pardir))

with open(os.path.join(basepath, "Topics/tutorial_supplementals/stopwords/en"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [24]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

In [25]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

58745
53040


##### Create Feature Remove Lists

In [ ]:
stopword_list = pre.find_stopwords(docterm_matrix, 100)
stopword_list

In [ ]:
hapax = set(docterm_matrix[docterm_matrix == 1].index)
#hapax = set(docterm_matrix[(docterm_matrix < 2).all(axis=1)].index)
len(hapax)

In [ ]:
hapax_list = collection.find_hapax(docterm_matrix)
len(hapax_list)

In [ ]:
feature_list = stopword_list.union(hapax_list)
print(feature_list)
clean_term_frequency = docterm_matrix.drop(feature_list)

##### Remove Features

In [ ]:
feature_list = stopword_list.union(hapax_list)
clean_term_frequency = collection.remove_features(docterm_matrix, feature_list)

##### Lemmata anhand POS-Tags auswählen

In [ ]:
lemmas = collection.filter_POS_tags(corpus_csv)
next(lemmas)

## Topic Modeling with Gensim

In [ ]:
corpusPath = "gb_plain.mm"

mm = MmCorpus(corpusPath) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen


In [ ]:
#in case you're only loading the corpus - build dict first

import pickle

pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [ ]:
id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [ ]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [ ]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [ ]:
model = LdaModel(corpus=mm, id2word=id_types, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen

In [ ]:
topic_nr_x = model.get_topic_terms(10)

[id_types[i[0]] for i in topic_nr_x]

In [ ]:
topics = model.show_topics(num_topics = 60)

In [ ]:
topics

## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [3]:
import os.path
basepath =  os.path.abspath(os.path.join(".", os.pardir))
basepath

'C:\\Users\\Philip\\Documents\\Python35-32_Scripts\\Funs\\DARIAH'

In [ ]:
#path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
path_to_mallet = "mallet"
path_to_corpus = os.path.join(basepath, "Topics\\corpus_txt")

malletBinary, outfolder = mal.create_mallet_binary(path_to_corpus, path_to_mallet)

#toDo absolute Pfade

##### Mallet output erzeugen

In [10]:
path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
malletBinary = "/home/sina/Uni/Dariah/DARIAH-Topics/Topics/mallet_output/malletBinary.mallet"
mal.create_mallet_model(malletBinary, outfolder, path_to_mallet)

24-Jan-2017 15:27:57 DEBUG mallet: mallet_output
24-Jan-2017 15:27:57 INFO mallet: Accessing Mallet ...
24-Jan-2017 15:28:12 DEBUG mallet: Mallet file available.


## Visualisierung 

In [ ]:
lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)

#### Heatmap

In [ ]:
heatmap = visual.make_heatmap()

In [ ]:
visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [ ]:
vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [ ]:
vis.make_interactive()

In [ ]:
vis.save_interactive("./visualizations/interactive")

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

In [ ]:
![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)